#Introduction to Databases & SQL



Most DSSG teams will store their data in a SQL (specifically PostgreSQL) database. SQL provides many advantages for our type of work, such as the ability to process large amounts of data efficiently. In this module, we will load and query our data on a SQL server. 

##Goals

- Learn SQL basics
- Be comfortable writing basic SQL queries

## Tasks
- Create schema
- Create tables for Chicago building permits and violations
- Copy data from Chicago's open-data portal to your tables
- View the first ten rows of your datasets
- 

##Tools
- psql (command line)
- dBeaver

##Basic database structure
- Database host: the computer on which the database is running. We use Amazon RDS as our host.
- Database: a self-contained set of tables and schema. A host can run many databases. This summer, we will operate databases for almost all projects from the same Amazon host.
- Schema: similar to a folder. A database can contain many schema, each containing many tables. 
- Tables: tables are like spreadsheets. They have rows and columns and values.

In this image, the database is `training_2015`, the schema we're investigating is `jwalsh`, and the tables in that schema are `building_violations` and `building_permits` (the host address does not appear):
![Tables stored in a schema stored in a database, as shown in dBeaver](img/db_schema_table.png)

To use those tables, you need to give the schema name and table name separated by a period, e.g. `jwalsh.building_violations`. Don't use periods in your schema, table, or column names! It will save you headaches.

##Create schema
```
create schema jwalsh;
```

##Create table
```
csvsql -i postgresql building_permits.csv
csvsql -i postgresql building_permits.csv | sed -E 's/\"//g' | tr [:upper:] [:lower:] 
csvsql -i postgresql building_violations.csv | sed -E 's/\"//g' | tr [:upper:] [:lower:] 
```

##Copy data
Clean and copy the building-permits dataset

```
cat building_permits.csv |
sed 's/\$//g' |
psql -h dssgsummer2014postgres.c5faqozfo86k.us-west-2.rds.amazonaws.com -U jwalsh -d training_2015 \
     -c "\COPY jwalsh.building_permits FROM STDIN WITH CSV HEADER;"
```

Clean and copy the building-violations dataset (we'll use a sample to avoid wi-fi delays):

```
wget -O- https://raw.githubusercontent.com/dssg/data-challenges/master/BuildingInspections/data/Building_Violations_sample_50000.csv > building_violations_sample.csv

cat building_violations_sample.csv | tr [:upper:] [:lower:] | csvsql -i postgresql | sed 's/\"//g' 

cat building_violations_sample.csv |
sed 's/, ,/,,/g' |
psql -h dssgsummer2014postgres.c5faqozfo86k.us-west-2.rds.amazonaws.com -U jwalsh -d training_2015 \
     -c "\COPY jwalsh.building_violations_sample FROM STDIN WITH CSV HEADER;"
```

Copy full dataset from jwalsh schema:

```
CREATE TABLE [schema].[table] AS (SELECT * FROM jwalsh.building_violations);
```

##Query data
Take a look at the first ten rows of each dataset:

```
SELECT * FROM jwalsh.building_violations LIMIT 10;
SELECT * FROM jwalsh.building_permits LIMIT 10;
```

How many rows are there?

```
SELECT COUNT(*) FROM jwalsh.building_violations AS a LIMIT 10;
```

Only look at the data that meet specified conditions:

```
SELECT * FROM jwalsh.building_violations AS a WHERE a.estimated_cost > 1000 LIMIT 10;
SELECT COUNT(*) FROM jwalsh.building_violations AS a WHERE a.estimated_cost > 1000;
```

Get conditional statistics by using an aggregate function (e.g. `AVG`) and `GROUP BY`. Sort using `ORDER BY` from top to bottom using `DESC`:

```
SELECT address, COUNT(*) AS freq FROM jwalsh.building_violations GROUP BY address ORDER BY freq DESC LIMIT 10;
SELECT violation_date, COUNT(*) AS freq FROM jwalsh.building_violations GROUP BY violation_date ORDER BY freq DESC LIMIT 6;
```

We can use aliases. Here is an example where `a` is a table alias and `min_date`, `max_date`, and `range` are column aliases: 

```
SELECT MIN(a.violation_date) AS min_date, MAX(a.violation_date) AS max_date, MAX(a.violation_date) - MIN(a.violation_date) as range FROM jwalsh.building_violations AS a;
```

We can use sub-queries:

```
SELECT a.* FROM (SELECT * FROM jwalsh.building_violations AS a LIMIT 10) AS a;
```

##Join data
```
SELECT * FROM jwalsh.building_permits AS a, jwalsh.building_violations AS b 
WHERE a.location = b.location 
LIMIT 10;
```



```
SELECT * 
FROM   (SELECT  *,
                street_number || ' ' || street_direction || ' ' || street_name || ' ' || suffix AS address
        FROM    jwalsh.building_permits) AS a
LEFT JOIN jwalsh.building_violations AS b ON a.location = b.location OR a.address = b.address
LIMIT 10;
```

Fuzzy matching:

```
SELECT * FROM ( SELECT *, 
                       ROUND(latitude,5) AS lat_rounded, 
                       ROUND(longitude,5) AS long_rounded 
                FROM   jwalsh.building_permits) AS a, 
              ( SELECT *,
                       ROUND(latitude,5) AS lat_rounded, 
                       ROUND(longitude,5) AS long_rounded 
                FROM   jwalsh.building_violations) AS b 
WHERE a.lat_rounded = b.lat_rounded AND a.long_rounded = b.long_rounded
LIMIT 10;
```

##Resources
- [Intro to SQL](http://software-carpentry.org/v4/databases/)
- [Associating lat-long with census tract using PostGIS](http://gis.stackexchange.com/questions/18559/bulk-lookup-of-address-census-tract-and-block)